In [23]:
import collections
import numpy as np
import pandas as pd
import os, sys
import re

from argparse import Namespace

In [80]:
args = Namespace(
    raw_train_dataset_csv="data/yelp/raw_train.csv",
    raw_test_dataset_csv="data/yelp/raw_test.csv",
    custom_dataset_csv="data/scalling_result400000-500000.csv",
    path_to_custom_dataset="..\\..\\..\\..\\RussianNLP\\public_youtube1120_mp3\\",
    preprocessed_dataset="./data/like_yelp_custom_ds.csv",
    proportion_subset_of_train=0.1,
    train_proportion=0.7,
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv="data/yelp/reviews_with_splits_lite.csv",
    seed=1337
)

In [82]:
def load_custom_dataset(data_path):
    df = pd.read_csv(data_path, index_col=0)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

cp_train_reviews = load_custom_dataset(args.custom_dataset_csv)
cp_train_reviews.head()

,0,1,2,3,4,5,path,emotions
0,0.171195,0.529780,0.162259,0.095235,0.040569,0.000963,.\public_youtube1120\4\88\5ac360108ae3.txt,neutral
1,0.047468,0.818532,0.048682,0.077543,0.007252,0.000523,.\public_youtube1120\4\88\5ac5f00a00e8.txt,neutral
2,0.135218,0.484933,0.246828,0.122521,0.009197,0.001304,.\public_youtube1120\4\88\5aeff595b96e.txt,negative
3,0.101549,0.574761,0.089627,0.136040,0.023855,0.074168,.\public_youtube1120\4\88\5bdbd9882ce0.txt,neutral
4,0.183379,0.430083,0.182002,0.200766,0.003571,0.000199,.\public_youtube1120\4\88\5bef40e49a15.txt,skip


In [115]:
import copy

# Delete left columns
col_for_delete = [str(i) for i in range(6)]
cp_train_reviews = cp_train_reviews.drop(columns=col_for_delete)

In [121]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^а-яёЁА-Яa-zA-Z.,!?]+", r" ", text)
    return text

def text_by_path(path):
    path = path[1:]
    path = "\\\\".join(path.split("\\"))
    try:
        with open(args.path_to_custom_dataset + path, 'r', encoding="utf-8") as file:
            text = file.read()
    except FileNotFoundError:
        return -1
    else:
        return preprocess_text(text)
        
cp_train_reviews.path = cp_train_reviews.path.apply(lambda x: text_by_path(x))

In [129]:
cp_train_reviews['split'] = ""
item_list = []
np.random.seed(args.seed)

np.random.shuffle(cp_train_reviews.values)
    
n_total = len(cp_train_reviews)
n_train = int(args.train_proportion * n_total)
n_val = int(args.val_proportion * n_total)
n_test = int(args.test_proportion * n_total)
    
# Give data point a split attribute
for _ in range(n_train):
    item_list.append('train')

for _ in range(n_train, n_train+n_val):
    item_list.append('val')

for _ in range(n_train+n_val, n_train+n_val+n_test):
    item_list.append('test')

In [132]:
cp_train_reviews['split'] = item_list

In [135]:
cp_train_reviews.head()

,path,emotions,split
0,здрасте варвара степановна,skip,train
1,надо было еще и голову отбить напрочь,positive,train
2,слушай а может нам развестись,neutral,train
3,это ресепшен передал,neutral,train
4,теперь не найти,neutral,train


In [134]:
cp_train_reviews.split.value_counts()

train    70000
test     15000
val      15000
Name: split, dtype: int64

In [ ]:
cp_train_reviews.to_csv(args.preprocessed_dataset)

In [16]:
final_reviews.to_csv(args.output_munged_csv, index=False)